In [1]:
from datetime import datetime, timedelta
import pandas as pd
import sqlite3
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import matplotlib.pyplot as plt
import ta
import yfinance as yf
import seaborn as sns
import numpy as np
from queue import Queue
import holidays
import scipy as sp
from itertools import combinations
from pymongo import MongoClient
import swifter

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

C:\Users\thede\AppData\Roaming\Python\Python38\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from helper.client import *
from helper.machinelearner import *
from helper.dbmgr import *
from helper.dataprocessor import *

# Create an instance of CoinbaseClient
client = CoinbaseClient(api_key, api_secret)

# Get and format data from CB

In [3]:
import pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["mydatabase"]
mycol = mydb["customers"]
mydict = { "name": "John", "address": "Highway 37" }

x = mycol.insert_one(mydict)

print(x.inserted_id)

65d5f3b75ed6b51afd22b89a


In [4]:
# unique_pids = client.get_unique_product_ids()
# unique_pids = [f"{i}-{'USDC'}" for i in unique_pids]

# # start_time = datetime(2022, 10, 1)
# start_time = datetime(2024, 2, 14)
# end_time = datetime(2024, 2, 15)

# db_manager = DatabaseManager(start_time=start_time, end_time=end_time)
# print(db_manager.check_row_counts())

# db_manager.main(unique_pids)

# Technical Indicator creation

In [5]:
db_uri = 'mongodb://localhost:27017/'  
db_name = 'crypto_data'  

fe = FeatureEngineer(db_name)
processed_data = fe.run()

Processing Collections:   0%|          | 0/6 [00:00<?, ?it/s]

Processing Collections: 100%|██████████| 6/6 [00:13<00:00,  2.26s/it]


# Preprocessing creation
- Labels
- Scaling
- Normalisation
- Stationary features

# Machine Learning on one set of parameters
- Grid Search for best parameters
- view feature importance on the best set and trim

# Save best model for backtesting

# Backtester